## Modelo de datos de empleados

#### Este modelo de datos es un modelo introductorio. Representa los detalles básicos de un empleado, como un alias único, nombre, apellido, designación, gerente y habilidades.

Este modelo de datos representa algunas técnicas como el manejo de atributos complejos, como tener más de una habilidad. Este modelo es también un ejemplo de relación uno a muchos a través del gerente y sus empleados de informes que se ha logrado mediante el índice secundario DirectReports.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un registro de empleado utilizando el alias de inicio de sesión del empleado, facilitada por una tabla llamada Employee.

* Buscar empleados por nombre, facilitado por el índice secundario global de la tabla Empleado llamado Name.

* Recuperación de todos los informes directos de un gerente utilizando el alias de inicio de sesión del gerente, facilitada por el índice secundario global de la tabla Empleado llamado DirectReports.

In [1]:
import pandas as pd
import dynamodb
from dynamodb import DynamoTable
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('Employees')
    print(dt)
except:
    dt.create_table(
        table_name='Employees',
        partition_key='LoginAlias',
        partition_key_type='S',
        provisioned=True,
)

- Table name: Employees            
- Table arn: arn:aws:dynamodb:us-east-1:557908831303:table/Employees            
- Table creation: 2023-03-07 10:36:34.422000-03:00            
- [{'AttributeName': 'LoginAlias', 'KeyType': 'HASH'}]            
- [{'AttributeName': 'FirstName', 'AttributeType': 'S'}, {'AttributeName': 'LastName', 'AttributeType': 'S'}, {'AttributeName': 'LoginAlias', 'AttributeType': 'S'}, {'AttributeName': 'ManagerLoginAlias', 'AttributeType': 'S'}]


In [ ]:
# Itireate over json file and add items to table
with open('songs.json') as json_file:
    songs = json.load(json_file)
songs_form = json.loads(json.dumps(songs), parse_float=Decimal)

with dt.table.batch_writer() as batch:
    for song in songs_form:
        batch.put_item(Item=song)

In [ ]:
dt.load_json("employees.json")

In [ ]:
dt.create_global_secondary_index(
    att_name="FirstName",
    att_type="S",
    sort_index="LastName",
    sort_type="S",
    i_name="NameIndex",
    i_rcu=5,
    i_wcu=5
)

In [ ]:
from time import sleep
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [ ]:
dt.create_global_secondary_index(
    att_name="ManagerLoginAlias",
    att_type="S",
    i_name="DirectReportsIndex2",
    proj_type=["FirstName", "LastName"],
    i_rcu=5,
    i_wcu=5
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [4]:
dt.list_gsi

['DirectReportsIndex2', 'NameIndex', 'DirectReportsIndex']

In [6]:
dt.check_status_gsi()

'ACTIVE'

In [7]:
dt.query_items("janed", to_pandas=True)

,LoginAlias,ManagerLoginAlias,FirstName,LastName,Skills
0,janed,marthar,Jane,Doe,[software]
